# Packages

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [4]:
# Load the uploaded file to examine its structure and contents and display it
file_path = 'Nat_Gas.csv'
nat_gas_data = pd.read_csv(file_path)

nat_gas_data.head()

,Dates,Prices
0,10/31/20,10.1
1,11/30/20,10.3
2,12/31/20,11.0
3,1/31/21,10.9
4,2/28/21,10.9


In [3]:
# Convert Dates column to datetime
nat_gas_data['Dates'] = pd.to_datetime(nat_gas_data['Dates'], format='%m/%d/%y')

# Define the pricing model function
def price_storage_contract(injection_dates, withdrawal_dates, prices, injection_rate, withdrawal_rate, max_volume, storage_costs):
    # Initialize variables
    volume = 0  # Current stored volume
    total_value = 0  # Contract value

    # Sort dates for processing
    all_dates = sorted(set(injection_dates + withdrawal_dates))
    
    for date in all_dates:
        # Check if it's an injection or withdrawal date
        if date in injection_dates:
            # Calculate injected volume and cost
            injected_volume = min(injection_rate, max_volume - volume)
            volume += injected_volume
            total_value -= injected_volume * prices[date]  # Subtract purchase cost
        elif date in withdrawal_dates:
            # Calculate withdrawn volume and profit
            withdrawn_volume = min(withdrawal_rate, volume)
            volume -= withdrawn_volume
            total_value += withdrawn_volume * prices[date]  # Add selling revenue
        
        # Subtract storage costs for the current volume
        total_value -= storage_costs * volume

    return total_value

# Prepare test parameters
prices_dict = dict(zip(nat_gas_data['Dates'], nat_gas_data['Prices']))
test_injection_dates = [datetime(2020, 11, 30), datetime(2020, 12, 31)]
test_withdrawal_dates = [datetime(2021, 1, 31), datetime(2021, 2, 28)]
test_injection_rate = 100  # units per day
test_withdrawal_rate = 80  # units per day
test_max_volume = 500  # maximum storage capacity
test_storage_costs = 0.1  # cost per unit volume per day

# Calculate contract value
contract_value = price_storage_contract(
    injection_dates=test_injection_dates,
    withdrawal_dates=test_withdrawal_dates,
    prices=prices_dict,
    injection_rate=test_injection_rate,
    withdrawal_rate=test_withdrawal_rate,
    max_volume=test_max_volume,
    storage_costs=test_storage_costs
)

contract_value

-432.0

The result of $432$ indicates that, in this scenario, there is a net loss of $432$ monetary units.